In [1]:
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import sys
import json
import pickle
import json

from tqdm import tqdm
import itertools

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

In [2]:
seeds = [999, 7, 42]
test_size = 0.3

In [ ]:

data_df = pd.read_csv('./../Data/1000_features_survival_3classes.csv',index_col=0).drop(['index', 'y'],axis=1)
data_df_event_time = data_df[['event', 'time']]

# Initialize the LabelEncoder
le = LabelEncoder()

# Loop through each column and apply encoding to object type columns
for col in data_df.columns:
    if data_df[col].dtype == 'object':
        data_df[col] = le.fit_transform(data_df[col])

data_df = data_df.fillna(data_df.mean())
data_df = data_df.loc[:, data_df.nunique() > 1]

# data_df = pd.get_dummies(data_df.drop(['event', 'time'], axis=1),dtype='int')
scaler = MinMaxScaler()
data_df = pd.DataFrame(scaler.fit_transform(data_df.drop(['event', 'time'], axis=1)), 
                       columns=data_df.drop(['event', 'time'], axis=1).columns)
data_df['event'] = [int(e) for e in data_df_event_time['event']]
data_df['time'] = data_df_event_time['time']

data_df = data_df.fillna(data_df.mean())
data_df = data_df.loc[:, data_df.nunique() > 1]

hyperopts_df = {'penalizer': [], 'l1_ratio': [], 'train_ci':[], 'valid_ci':[], 'test_ci':[], 'time_elapsed':[], 
                'train_ci_ls':[], 'valid_ci_ls':[], 'test_ci_ls':[]}

combinations = [i for i in itertools.product([0.01, 1], [0.5, 1])]
ncombinations = len(combinations)

for penalizer, l1_ratio in tqdm():
    hyperopts_df['penalizer'] = hyperopts_df['penalizer'] + [penalizer]
    hyperopts_df['l1_ratio'] = hyperopts_df['l1_ratio'] + [l1_ratio]
    train_ci_ls = []
    valid_ci_ls = []
    test_ci_ls = []
    elapsed_time_ls = []
    for seed in seeds:
        
        
        data_train, data_tmp = train_test_split(data_df, test_size=test_size, random_state=seed)
        data_valid, data_test = train_test_split(data_tmp, test_size=0.5, random_state=seed)
        
        cph = CoxPHFitter(penalizer=penalizer, l1_ratio=l1_ratio)
        start = time.time()
        cph.fit(data_train, duration_col='time', event_col='event')
        end = time.time()
        
        elapsed_time_ls = elapsed_time_ls + [end-start]
        train_ci_ls = train_ci_ls + [cph.score(data_train, scoring_method='concordance_index')]
        valid_ci_ls = valid_ci_ls + [cph.score(data_valid, scoring_method='concordance_index')]
        test_ci_ls = test_ci_ls + [cph.score(data_test, scoring_method='concordance_index')]

    print("\nTrain: ",train_ci_ls, "\nValid: ",valid_ci_ls, "\nTest: ",test_ci_ls,
          "\nElapsed time: ", elapsed_time_ls)

    hyperopts_df['train_ci'] = hyperopts_df['train_ci'] + [np.mean(train_ci_ls)]
    hyperopts_df['valid_ci'] = hyperopts_df['valid_ci'] + [np.mean(valid_ci_ls)]
    hyperopts_df['test_ci'] = hyperopts_df['test_ci'] + [np.mean(test_ci_ls)]
    hyperopts_df['time_elapsed'] = hyperopts_df['time_elapsed'] + [np.mean(elapsed_time_ls)]
    hyperopts_df['train_ci_ls'] = hyperopts_df['train_ci_ls'] + [train_ci_ls]
    hyperopts_df['valid_ci_ls'] = hyperopts_df['valid_ci_ls'] + [valid_ci_ls]
    hyperopts_df['test_ci_ls'] = hyperopts_df['test_ci_ls'] + [test_ci_ls]
    # cph.print_summary()

hyperopts_df['seeds'] = [seeds]*ncombinations
pd.DataFrame(hyperopts_df)

In [21]:
hyperopts_df = pd.DataFrame(hyperopts_df)
hyperopts_df.to_csv('./CPH/results/hyperopts_results.df')

In [22]:
pd.read_csv('./CPH/results/hyperopts_results.df',index_col=0).sort_values('valid_ci')

,penalizer,l1_ratio,train_ci,valid_ci,test_ci,time_elapsed,train_ci_ls,valid_ci_ls,test_ci_ls,seeds
0,0.01,0.5,0.963063,0.701909,0.697117,178.488625,"[0.9618255491576029, 0.9641194193371679, 0.963...","[0.7118337850045167, 0.616504854368932, 0.7773...","[0.6545842217484008, 0.7289760348583878, 0.707...","[999, 7, 42]"
1,0.01,1.0,0.926156,0.701994,0.720666,309.364704,"[0.9275868085848892, 0.9260897962623518, 0.924...","[0.7276422764227642, 0.6954986760812003, 0.682...","[0.7313432835820896, 0.787363834422658, 0.6432...","[999, 7, 42]"
3,1.00,1.0,0.786743,0.733135,0.672233,246.550716,"[0.7858236879350123, 0.7755304131639382, 0.798...","[0.7005420054200542, 0.7294792586054722, 0.769...","[0.6652452025586354, 0.6570806100217865, 0.694...","[999, 7, 42]"
2,1.00,0.5,0.786770,0.733302,0.672233,276.970899,"[0.7858236879350123, 0.7755093441206835, 0.798...","[0.7005420054200542, 0.7294792586054722, 0.769...","[0.6652452025586354, 0.6570806100217865, 0.694...","[999, 7, 42]"
